## **1) Processi Totalmente Ignari:**

In questo caso, i processi non sono consapevoli l'uno dell'altr oe operano indipendentemente senza alcuna comunicazione diretta o scambio di informazioni. Esempio: Parallelismo di dati.
Supponiamo di avere un'immagine divisa in pixel. Ogni processo si occupa di elaborare i dati in un determinato set di pixel senza avere alcuna consapevolezza degli altri processi che elaborano altri pixel.

In [ ]:
from multiprocessing import Process
"""
multiprocessing è una libreria per la creazione, la comunicazione e la sincronizzazione
tra processi nella programmazione parallela e concorrente
Process è una classe per creare procesi eseguendo una funzione (o metodo) specificata come target.
"""
# Funzione per processare i dati ovvero prendo i dati in ingresso li moltiplico per 2
# e li restituisco
def process_funcion(data):
    result=data*2
    print(result)

if __name__ == "__main__":
    data_list= [1, 2, 3, 4]
    processes = []

    for data in data_list:
        p = Process(target=process_funcion, args=(data,))
        # aggiungo p alla lista processes 
        processes.append(p)
        p.start() # viene avviata l'esecuzione del processo separato
        
    for p in processes:
        p.join() # blocca il processo principale fino a quando il processo separato non termina

## **2) Processi Indireattamente a Conoscenza Uno dell'Altro:**

In questo caso, i processi potrebbero non conoscere direttametne gli altri, ma possono interagire attraverso uno scambio indiretto di informazioni, ad esempio, attraveso una strutta dati condivisa o un intermediario. Esempio: Una coda convisa. I processi possono mettere dati in una coda condivisa e prelevarli successivamente. Anche se i processi non conoscono direttamente gli altri processi, condividono una struttura dati (la coda) per scambiare informazioni.

In [ ]:
from asyncio import Queue
from multiprocessing import Process, current_process, Pipe
import os
"""
-- Queue: classe che rappresenta una coda condivisa tra procesi.
E' utilizzata per consitire la comunicazione tra processi,
consentendo loro di scambiarsi dati in modo sicuro.
put(item): Aggiunge un elemetno alla coda.
get(): Rimuove e restituisce un elemtno dalla coda.
empty(): Restituisce True se la coda è vuota, altrimente restitusice False.
full(): Restitusce True se la coda è piena, altrimenti restituesce False.
qsize(): Restituisce il numero di leemtni presenti nella coda.
close(): Chiude l coda.
-- curren_process: funzioen che restituisce un oggetto Process
che rappresenta il processo in esecuzione.
"""

def process_id():
    # stampo il pid del server, il nome del processo e il suo pid
    print(f"Server PID: {os.getpid()}, Process Name: {current_process().name}, Process PID: { current_process().pid}")


def process_funcion(data, result_queue):
    process_id()
    print("Elabora: ", data)
    result = data * 2
    result_queue.put(result)

if __name__=="__main__":
    # si entra nel main
    data_list = [1, 2, 3, 4]
    # viene creata una lista di dati
    result_queue=Queue()
    processes = []

    for data in data_list:
        p= Process(target=process_funcion, args=(data, result_queue))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    print("Il main stampa i risultati")
    while not result_queue.empty():
        result= result_queue.get()
        print(result)

## **3) Processi diretatmente a conoscenza:**

In questo caso, i processi sono consapevoli l'uno dell'altro e possono comunicare diretamente. Esempio: Comunicazione diretta. I processi comunicano utilizzando messaggi diretti o meccanismi di comunicazione. Ad esempio, uno scenario scambio di dati tra server e clietn o tra processi all'inerno di un applicazion. Adesemipo, uno scenario di scambio did dati tra server e client o processi all'inerno di un'applicazione che utilizza canali di comunicazione diretti.

In [ ]:
import os # il modulo fornisce funzionalità per interagire con il sistema operativo
from multiprocessing import Process, current_process, Pipe

def process_id():
    print(f"Server PID: {os.getpid()}, Process Name: {current_process().name}, Process PID. { current_process().pid}")

def process_funcion(conn):
    process_id()
    print("Elaboro il dato ricevuto ed invio il risultato")
    data_recived = conn.recv()
    result = data_recived * 2
    conn.send(result)
    conn.close()

if __name__=="__main__":
    process_id()
    print("Creo una connessione e un processo figlio")
    parent_conn, child_conn = Pipe()
    data = 5
    p = Process(target=process_funcion, args=(child_conn,))
    p.start()
    parent_conn.send(data)
    # result è il risultato che viene preso con recv tramite conn genitore
    result = parent_conn.recv()
    p.join
    print("Stampo il risultato ricevuto")
    print(result)